# Model Training

In [ ]:
#Basic libraries
import os
import time
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


#Deep Learning libraries
import keras
from keras.layers import Dense, Activation,Dropout, BatchNormalization,LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras.activations import sigmoid
from keras.losses import binary_crossentropy
import talos as ta


#Machine Learning libraries
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

## Create function to create ROC curves per patient
In order to correctly compare the models we want to see how many patients do they succesfully predict. To do that predictions are grouped each 45s (45 points), and increasing thresholds are used to see how do they perform. `any()` method is used as if the model predicts even for 1s that output is positive we take that it predicted positive patient. It is worthy to say that if `all()` method is used instead results are pretty much the same.

In [ ]:
#inputs are data (predicted probability) and group (actual class)
def per_patient(data,group):
    
    
    #initialise lists for tpr and fpr
    tpr=[]
    fpr=[]
    
    
    #iterare over 101 thresholds (less would be enough but it is just to make sure)
    for j in range(101):
        
        
        #initialise prediction lists (to be filled with True or False)
        out1=[]
        out2=[]
        
        
        #iterate over each patient
        for i in range(int(len(data)/45)):
            
            
            #for first iteration (threshold=0) greater or equal condition is needed
            if j==0:
                
                
                #apend to out1 predicted class and to out2 actual class
                out1.append((data[i*45:45*(i+1)]>=0).any())
                out2.append((np.asarray(group[i*45:45*(i+1)])==1).any())
                
                
            #all other iterations greater condition is used
            else:
                out1.append((data[i*45:45*(i+1)]>j/100).any())
                out2.append((np.asarray(group[i*45:45*(i+1)])==1).any())
                
                
        #create confusion matrix and append tpr and fpr to lists
        cf=pd.DataFrame(confusion_matrix(pd.DataFrame(out2),pd.DataFrame(out1)))
        tpr.append(cf.iloc[0,0]/(cf.iloc[0,0]+cf.iloc[0,1]))
        fpr.append(cf.iloc[1,0]/(cf.iloc[1,0]+cf.iloc[1,1]))
        
        
    #return tpr and fpr values for increasing thresholds
    return tpr,fpr

## Load data

In [ ]:
#train data
train_X_bis=pd.read_csv('data\Data_model\\train_X_BIS.csv',header=None,index_col=False).values
train_X_mov=pd.read_csv('data\Data_model\\train_X_MOV.csv',header=None,index_col=False).values
train_X_nibp=pd.read_csv('data\Data_model\\train_X_NIBP.csv',header=None,index_col=False).values

train_y_bis=pd.read_csv('data\Data_model\\train_y_BIS.csv',header=None,index_col=False)[0].tolist()
train_y_mov=pd.read_csv('data\Data_model\\train_y_MOV.csv',header=None,index_col=False)[0].tolist()
train_y_nibp=pd.read_csv('data\Data_model\\train_y_NIBP.csv',header=None,index_col=False)[0].tolist()


#test data
test_X_bis=pd.read_csv('data\Data_model\\test_X_BIS.csv',header=None,index_col=False).values
test_X_mov=pd.read_csv('data\Data_model\\test_X_MOV.csv',header=None,index_col=False).values
test_X_nibp=pd.read_csv('data\Data_model\\test_X_NIBP.csv',header=None,index_col=False).values

test_y_bis=pd.read_csv('data\Data_model\\test_y_BIS.csv',header=None,index_col=False)[0].tolist()
test_y_mov=pd.read_csv('data\Data_model\\test_y_MOV.csv',header=None,index_col=False)[0].tolist()
test_y_nibp=pd.read_csv('data\Data_model\\test_y_NIBP.csv',header=None,index_col=False)[0].tolist()

## Reshape LSTM data

In [ ]:
#fuction to reshape data to match LSTM requirements
def to_LSTM(x_data, y_data):
    
    
    #initialise X and y
    out=[]
    inp=[]
    
    
    #iterate over patients (45s/patient)
    for i in range(int(x_data.shape[0]/45)):
        
        
        #reshape X (per patient)
        X=(pd.DataFrame(x_data).iloc[45*i:45*(i+1),]).values
        inp.append(X.reshape((X.shape[0], 1,X.shape[1])))
        
        
        #turn y to 1 point per patient
        if (pd.DataFrame(y_data).iloc[i*45:(i+1)*45]==1).any()[0]:
            out.append(1)
        else:
            out.append(0)

    
    #reshape X (overall)
    inp=np.array(inp)
    inp=inp.reshape((inp.shape[0],inp.shape[1],inp.shape[3]))
    
    
    #output X and y
    return inp, out

In [ ]:
#rehsape X and y data
train_X_LSTM_bis,train_y_LSTM_bis = to_LSTM(train_X_bis,train_y_bis)
test_X_LSTM_bis,test_y_LSTM_bis = to_LSTM(test_X_bis,test_y_bis)

train_X_LSTM_mov,train_y_LSTM_mov = to_LSTM(train_X_mov,train_y_mov)
test_X_LSTM_mov,test_y_LSTM_mov = to_LSTM(test_X_mov,test_y_mov)

train_X_LSTM_nibp,train_y_LSTM_nibp = to_LSTM(train_X_nibp,train_y_nibp)
test_X_LSTM_nibp,test_y_LSTM_nibp = to_LSTM(test_X_nibp,test_y_nibp)

# Prediction
Models are created according to best parameters and trained. Then predictions for test set are made and its ROC curves calculated as well as area under curve (AUC).

## BIS 

In [ ]:
#RF
model_rf_bis = RandomForestClassifier(n_estimators=200,min_samples_split=5,min_samples_leaf=4,max_depth=10,n_jobs=-1,random_state=42)
model_rf_bis.fit(train_X_bis, train_y_bis)
pred_rf_bis = model_rf_bis.predict_proba(test_X_bis)
tpr_rf_bis,fpr_rf_bis=per_patient(pred_rf_bis[:,1],test_y_bis)
auc_rf_bis = auc(fpr_rf_bis, tpr_rf_bis)


#kNN
model_knn_bis = KNeighborsClassifier(n_neighbors=740,weights='distance',algorithm='auto',leaf_size=5,n_jobs=-1)
model_knn_bis.fit(train_X_bis, train_y_bis)
pred_knn_bis = model_knn_bis.predict_proba(test_X_bis)
tpr_knn_bis,fpr_knn_bis=per_patient(pred_knn_bis[:,1],test_y_bis)
auc_knn_bis = auc(fpr_knn_bis, tpr_knn_bis)


#SVC
model_svc_bis = SVC(C=10,kernel='rbf',gamma='auto',probability=True,random_state=42)
model_svc_bis.fit(train_X_bis, train_y_bis)
pred_svc_bis = model_svc_bis.predict_proba(test_X_bis)
tpr_svc_bis,fpr_svc_bis=per_patient(pred_svc_bis[:,1],test_y_bis)
auc_svc_bis = auc(fpr_svc_bis, tpr_svc_bis)

In [ ]:
#NN
num_neurons_bis=80
model_nn_bis=Sequential()
model_nn_bis.add(Dense(num_neurons_bis,input_dim=59,activation='sigmoid')) #INPUT DIM
model_nn_bis.add(Dropout(0.2)) #avoid overfitting
model_nn_bis.add(Dense(num_neurons_bis,activation='sigmoid'))
model_nn_bis.add(Dropout(0.2))
model_nn_bis.add(Dense(num_neurons_bis,activation='sigmoid'))
model_nn_bis.add(Dropout(0.2))
model_nn_bis.add(Dense(1,activation='sigmoid',kernel_initializer='normal'))
model_nn_bis.compile(optimizer=Adam(),loss='binary_crossentropy',metrics=['acc']) #lr=ta.utils.lr_normalizer(5,Adam)
model_nn_bis.fit(train_X_bis, train_y_bis, epochs=10, batch_size=16,
                        validation_data=(test_X_bis, test_y_bis), verbose=0, shuffle=True)
pred_nn_bis=model_nn_bis.predict(test_X_bis)
tpr_nn_bis,fpr_nn_bis=per_patient(pred_nn_bis,test_y_bis)
auc_nn_bis = auc(fpr_nn_bis, tpr_nn_bis)

In [ ]:
#LSTM
num_neurons_lstm_bis=35
model_lstm_bis=Sequential()
model_lstm_bis.add(LSTM(num_neurons_lstm_bis, input_shape=(train_X_LSTM_bis.shape[1], train_X_LSTM_bis.shape[2]),
                           activation=sigmoid,kernel_initializer='normal',dropout=0.1))
model_lstm_bis.add(Dense(1, activation=sigmoid,kernel_initializer='normal'))
model_lstm_bis.compile(loss=binary_crossentropy,optimizer=Adam(),metrics=['acc']) #lr=ta.utils.lr_normalizer(3.5,Adam)
model_lstm_bis.fit(train_X_LSTM_bis, train_y_LSTM_bis, epochs=300, batch_size=64,
                        validation_data=(test_X_LSTM_bis, test_y_LSTM_bis), verbose=0, shuffle=True)
pred_bis=model_lstm_bis.predict(test_X_LSTM_bis)
pred_lstm_bis=[l[0] for l in pred_bis]
fpr_lstm_bis, tpr_lstm_bis, threshold = roc_curve(test_y_LSTM_bis, pred_lstm_bis)
auc_lstm_bis = auc(fpr_lstm_bis, tpr_lstm_bis)

In [ ]:
#store BIS results
pd.DataFrame.from_dict({'tpr':tpr_rf_bis+tpr_knn_bis+tpr_svc_bis+tpr_nn_bis+list(tpr_lstm_bis),
             'fpr':fpr_rf_bis+fpr_knn_bis+fpr_svc_bis+fpr_nn_bis+list(fpr_lstm_bis),
             'model':['RF']*len(tpr_rf_bis)+['kNN']*len(tpr_knn_bis)+['SVC']*len(tpr_svc_bis)+['NN']*len(tpr_nn_bis)+['LSTM']*len(tpr_lstm_bis)}).to_csv(
'results\BIS_roc.csv',index=False)
pd.DataFrame.from_dict({'auc':[auc_rf_bis,auc_knn_bis,auc_svc_bis,auc_nn_bis,auc_lstm_bis],
                       'model':['RF','kNN','SVC','NN','LSTM']}).to_csv(
'results\BIS_auc.csv',index=False)

## MOV

In [ ]:
#RF
model_rf_mov = RandomForestClassifier(n_estimators=400,max_depth=30,min_samples_split=10,min_samples_leaf=2,n_jobs=-1,random_state=42)
model_rf_mov.fit(train_X_mov, train_y_mov)
pred_rf_mov = model_rf_mov.predict_proba(test_X_mov)
tpr_rf_mov,fpr_rf_mov=per_patient(pred_rf_mov[:,1],test_y_mov)
auc_rf_mov = auc(fpr_rf_mov, tpr_rf_mov)


#kNN
model_knn_mov = KNeighborsClassifier(n_neighbors=100,weights='uniform',algorithm='auto',leaf_size=5,n_jobs=-1)
model_knn_mov.fit(train_X_mov, train_y_mov)
pred_knn_mov = model_knn_mov.predict_proba(test_X_mov)
tpr_knn_mov,fpr_knn_mov=per_patient(pred_knn_mov[:,1],test_y_mov)
auc_knn_mov = auc(fpr_knn_mov, tpr_knn_mov)


#SVC
model_svc_mov = SVC(C=0.1,kernel='rbf',gamma='scale',probability=True,random_state=42)
model_svc_mov.fit(train_X_mov, train_y_mov)
pred_svc_mov = model_svc_mov.predict_proba(test_X_mov)
tpr_svc_mov,fpr_svc_mov=per_patient(pred_svc_mov[:,1],test_y_mov)
auc_svc_mov = auc(fpr_svc_mov, tpr_svc_mov)

In [ ]:
#NN
num_neurons_mov=100
model_nn_mov=Sequential()
model_nn_mov.add(Dense(num_neurons_mov,input_dim=59,activation='sigmoid'))
model_nn_mov.add(Dropout(0.1)) #avoid overfitting
model_nn_mov.add(Dense(num_neurons_mov,activation='sigmoid'))
model_nn_mov.add(Dropout(0.1))
model_nn_mov.add(Dense(1,activation='sigmoid',kernel_initializer='normal'))
model_nn_mov.compile(optimizer=Adam(),loss='binary_crossentropy',metrics=['acc']) #lr=ta.utils.lr_normalizer(3.5,Adam)
model_nn_mov.fit(train_X_mov, train_y_mov, epochs=50, batch_size=16,
                        validation_data=(test_X_mov, test_y_mov), verbose=0, shuffle=True)
pred_nn_mov=model_nn_mov.predict(test_X_mov)
tpr_nn_mov,fpr_nn_mov=per_patient(pred_nn_mov,test_y_mov)
auc_nn_mov = auc(fpr_nn_mov, tpr_nn_mov)

In [ ]:
#LSTM
num_neurons_lstm_mov=75
model_lstm_mov=Sequential()
model_lstm_mov.add(LSTM(num_neurons_lstm_mov, input_shape=(train_X_LSTM_mov.shape[1], train_X_LSTM_mov.shape[2]),
                           activation=sigmoid,kernel_initializer='normal',dropout=0.2))
model_lstm_mov.add(Dense(1, activation=sigmoid,kernel_initializer='normal'))
model_lstm_mov.compile(loss=binary_crossentropy,optimizer=Adam(),metrics=['acc']) #lr=ta.utils.lr_normalizer(5,Adam)
model_lstm_mov.fit(train_X_LSTM_mov, train_y_LSTM_mov, epochs=300, batch_size=64,
                        validation_data=(test_X_LSTM_mov, test_y_LSTM_mov), verbose=0, shuffle=True)
pred_mov=model_lstm_mov.predict(test_X_LSTM_mov)
pred_lstm_mov=[l[0] for l in pred_mov]
fpr_lstm_mov, tpr_lstm_mov, threshold = roc_curve(test_y_LSTM_mov, pred_lstm_mov)
auc_lstm_mov = auc(fpr_lstm_mov, tpr_lstm_mov)

In [ ]:
#store MOV results
pd.DataFrame.from_dict({'tpr':tpr_rf_mov+tpr_knn_mov+tpr_svc_mov+tpr_nn_mov+list(tpr_lstm_mov),
             'fpr':fpr_rf_mov+fpr_knn_mov+fpr_svc_mov+fpr_nn_mov+list(fpr_lstm_mov),
             'model':['RF']*len(tpr_rf_mov)+['kNN']*len(tpr_knn_mov)+['SVC']*len(tpr_svc_mov)+['NN']*len(tpr_nn_mov)+['LSTM']*len(tpr_lstm_mov)}).to_csv(
'results\MOV_roc.csv',index=False)
pd.DataFrame.from_dict({'auc':[auc_rf_mov,auc_knn_mov,auc_svc_mov,auc_nn_mov,auc_lstm_mov],
                       'model':['RF','kNN','SVC','NN','LSTM']}).to_csv(
'results\MOV_auc.csv',index=False)

## NIBP

In [ ]:
#RF
model_rf_nibp = RandomForestClassifier(n_estimators=100,min_samples_split=5,min_samples_leaf=2,max_depth=10,n_jobs=-1,random_state=42)
model_rf_nibp.fit(train_X_nibp, train_y_nibp)
pred_rf_nibp = model_rf_nibp.predict_proba(test_X_nibp)
tpr_rf_nibp,fpr_rf_nibp=per_patient(pred_rf_nibp[:,1],test_y_nibp)
auc_rf_nibp = auc(fpr_rf_nibp, tpr_rf_nibp)


#kNN
model_knn_nibp = KNeighborsClassifier(n_neighbors=100,weights='distance',algorithm='auto',leaf_size=5,n_jobs=-1)
model_knn_nibp.fit(train_X_nibp, train_y_nibp)
pred_knn_nibp = model_knn_nibp.predict_proba(test_X_nibp)
tpr_knn_nibp,fpr_knn_nibp=per_patient(pred_knn_nibp[:,1],test_y_nibp)
auc_knn_nibp = auc(fpr_knn_nibp, tpr_knn_nibp)


#SVC
model_svc_nibp = SVC(C=10,kernel='rbf',gamma='auto',probability=True,random_state=42)
model_svc_nibp.fit(train_X_nibp, train_y_nibp)
pred_svc_nibp = model_svc_nibp.predict_proba(test_X_nibp)
tpr_svc_nibp,fpr_svc_nibp=per_patient(pred_svc_nibp[:,1],test_y_nibp)
auc_svc_nibp = auc(fpr_svc_nibp, tpr_svc_nibp)

In [ ]:
#NN
num_neurons_nibp=90
model_nn_nibp=Sequential()
model_nn_nibp.add(Dense(num_neurons_nibp,input_dim=59,activation='sigmoid'))
model_nn_nibp.add(Dropout(0.2)) #avoid overfitting
model_nn_nibp.add(Dense(num_neurons_nibp,activation='sigmoid'))
model_nn_nibp.add(Dropout(0.2))
model_nn_nibp.add(Dense(num_neurons_nibp,activation='sigmoid'))
model_nn_nibp.add(Dropout(0.2))
model_nn_nibp.add(Dense(1,activation='sigmoid',kernel_initializer='normal'))
model_nn_nibp.compile(optimizer=Adam(),loss='binary_crossentropy',metrics=['acc']) #lr=ta.utils.lr_normalizer(5,Adam)
model_nn_nibp.fit(train_X_nibp, train_y_nibp, epochs=10, batch_size=16,
                        validation_data=(test_X_nibp, test_y_nibp), verbose=0, shuffle=True)
pred_nn_nibp=model_nn_nibp.predict(test_X_nibp)
tpr_nn_nibp,fpr_nn_nibp=per_patient(pred_nn_nibp,test_y_nibp)
auc_nn_nibp = auc(fpr_nn_nibp, tpr_nn_nibp)

In [ ]:
#LSTM
num_neurons_lstm_nibp=75
model_lstm_nibp=Sequential()
model_lstm_nibp.add(LSTM(num_neurons_lstm_nibp, input_shape=(train_X_LSTM_nibp.shape[1], train_X_LSTM_nibp.shape[2]),
                           activation=sigmoid,kernel_initializer='normal',dropout=0.1))
model_lstm_nibp.add(Dense(1, activation=sigmoid,kernel_initializer='normal'))
model_lstm_nibp.compile(loss=binary_crossentropy,optimizer=Adam(),metrics=['acc']) #lr=ta.utils.lr_normalizer(3.5,Adam)
model_lstm_nibp.fit(train_X_LSTM_nibp, train_y_LSTM_nibp, epochs=200, batch_size=16,
                        validation_data=(test_X_LSTM_nibp, test_y_LSTM_nibp), verbose=0, shuffle=True)
pred_nibp=model_lstm_nibp.predict(test_X_LSTM_nibp)
pred_lstm_nibp=[l[0] for l in pred_nibp]
fpr_lstm_nibp, tpr_lstm_nibp, threshold = roc_curve(test_y_LSTM_nibp, pred_lstm_nibp)
auc_lstm_nibp = auc(fpr_lstm_nibp, tpr_lstm_nibp)

In [ ]:
#store NIBP results
pd.DataFrame.from_dict({'tpr':tpr_rf_nibp+tpr_knn_nibp+tpr_svc_nibp+tpr_nn_nibp+list(tpr_lstm_nibp),
             'fpr':fpr_rf_nibp+fpr_knn_nibp+fpr_svc_nibp+fpr_nn_nibp+list(fpr_lstm_nibp),
             'model':['RF']*len(tpr_rf_nibp)+['kNN']*len(tpr_knn_nibp)+['SVC']*len(tpr_svc_nibp)+['NN']*len(tpr_nn_nibp)+['LSTM']*len(tpr_lstm_nibp)}).to_csv(
'results\\NIBP_roc.csv',index=False)
pd.DataFrame.from_dict({'auc':[auc_rf_nibp,auc_knn_nibp,auc_svc_nibp,auc_nn_nibp,auc_lstm_nibp],
                       'model':['RF','kNN','SVC','NN','LSTM']}).to_csv(
'results\\NIBP_auc.csv',index=False)